In [1]:
# Please note that this code just follows the provided video
# Mount data from drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Folder name
folderName = 'CS685 Project/Sanity Check'
assert folderName is not None, "[Error] Please enter folder name."


# Load python files from our folder
import sys
sys.path.append('/content/drive/My Drive/{}'.format(folderName))

%cd /content/drive/My\ Drive/$folderName/

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/176_V3hUNBvwdqw9WeYJjHSQNn8fY7Q2K/CS685 Project/Sanity Check


In [2]:
!pip install sentencepiece
!pip install transformers
# !pip install torch
!pip install rich[jupyter]

# Importing libraries
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import os

# Importing the T5 modules from huggingface/transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration

# rich: for a better display on terminal
from rich.table import Column, Table
from rich import box
from rich.console import Console

Paraphrase Shakespeare sentences using PEGASUS fine tuned for paraphrasing

https://huggingface.co/tuner007/pegasus_paraphrase


In [3]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

def get_response(input_text,num_return_sequences,num_beams):
  batch = tokenizer([input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=60,num_beams=num_beams, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

Downloading:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

In [4]:
non_parallel_path = "data/Shakespearean.csv"
non_parallel_df = pd.read_csv(non_parallel_path)

In [5]:
non_parallel_df.rename(columns={"0":"text"}, inplace=True)

In [6]:
non_parallel_df

,text
0,I have a mind to strike thee ere thou speak'st .
1,"Yet if thou say Antony lives , is well , Or fr..."
2,"Madam , he's well ."
3,Well said .
4,And friends with Caesar .
...,...
18390,He's in his fit now and does not talk after th...
18391,He shall taste of my bottle .
18392,"If he have never drunk wine afore , it will go..."
18393,"If I can recover him and keep him tame , I wil..."


In [7]:
context = "I do fear, the people Choose Caesar for their king."
get_response(context, num_return_sequences=5, num_beams=5)

['The people choose Caesar for their king.',
 'I am afraid that the people will choose Caesar for their king.',
 'I fear that the people will choose Caesar.',
 'I fear that the people will choose Caesar for their king.',
 'I fear that people will choose Caesar for their king.']

In [10]:
pseudo_parallel_df = pd.DataFrame(columns=["input", "target"])
pseudo_parallel_df

,input,target


In [11]:
for idx, context in non_parallel_df.iterrows():
  print("paraphrasing instance #" + str(idx))
  target = get_response(context[0], num_return_sequences=5, num_beams=5)
  target.sort(key=len)  
  pseudo_parallel_df.loc[idx] = [target[-1], context[0]]

Streaming output truncated to the last 5000 lines.
paraphrasing instance #13395
paraphrasing instance #13396
paraphrasing instance #13397
paraphrasing instance #13398
paraphrasing instance #13399
paraphrasing instance #13400
paraphrasing instance #13401
paraphrasing instance #13402
paraphrasing instance #13403
paraphrasing instance #13404
paraphrasing instance #13405
paraphrasing instance #13406
paraphrasing instance #13407
paraphrasing instance #13408
paraphrasing instance #13409
paraphrasing instance #13410
paraphrasing instance #13411
paraphrasing instance #13412
paraphrasing instance #13413
paraphrasing instance #13414
paraphrasing instance #13415
paraphrasing instance #13416
paraphrasing instance #13417
paraphrasing instance #13418
paraphrasing instance #13419
paraphrasing instance #13420
paraphrasing instance #13421
paraphrasing instance #13422
paraphrasing instance #13423
paraphrasing instance #13424
paraphrasing instance #13425
paraphrasing instance #13426
paraphrasing instance

In [12]:
pseudo_parallel_df

,input,target
0,I have a plan to speak to thee.,I have a mind to strike thee ere thou speak'st .
1,"If Antony lives, is friends with Caesar, or is...","Yet if thou say Antony lives , is well , Or fr..."
2,"He is well, Madame.","Madam , he's well ."
3,"Well said, that's right.",Well said .
4,They were friends with Caesar.,And friends with Caesar .
...,...,...
18390,He doesn't talk after the wisest because he's ...,He's in his fit now and does not talk after th...
18391,He will like the taste of my bottle.,He shall taste of my bottle .
18392,"If he has never drunk wine before, it will go ...","If he have never drunk wine afore , it will go..."
18393,I will not take much for him if I can recover ...,"If I can recover him and keep him tame , I wil..."


Store Paraphrases as X and Shakespeare sentences as Z

Resulting dataset: (X, Z)

Task for dataset: Given Paraphrase X generate original sentence Z


In [13]:
pseudo_parallel_df.to_csv("pseudo_parallel.csv")

Fine-tune T5 for this task